<a href="https://colab.research.google.com/github/Sagar10-9402/Crug/blob/main/custom_anootations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!pip install spacy-transformers
!python -m spacy download en_core_web_trf
!!pip install tika

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 27.6 MB/s 
     |████████████████████████████████| 1.2 MB 54.0 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 55.4 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.1
    Uninstalling spacy-3.4.1:
      Successfully uninstalled spacy-3.4.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 22.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.3/460.3 MB 3.3 MB/s eta 0:00:

['Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/',
 'Collecting tika',
 '  Downloading tika-1.24.tar.gz (28 kB)',
 '  Preparing metadata (setup.py) ... \x1b[?25l\x1b[?25hdone',
 'Requirement already satisfied: setuptools in /usr/local/lib/python3.7/dist-packages (from tika) (65.5.0)',
 'Requirement already satisfied: requests in /usr/local/lib/python3.7/dist-packages (from tika) (2.23.0)',
 'Requirement already satisfied: idna<3,>=2.5 in /usr/local/lib/python3.7/dist-packages (from requests->tika) (2.10)',
 'Requirement already satisfied: chardet<4,>=3.0.2 in /usr/local/lib/python3.7/dist-packages (from requests->tika) (3.0.4)',
 'Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.7/dist-packages (from requests->tika) (2022.9.24)',
 'Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /usr/local/lib/python3.7/dist-packages (from requests->tika) (1.24.3)',
 'Building wheels for collected p

In [3]:
import os 
import json
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import spacy 
from spacy.tokens import DocBin
from tqdm import tqdm
import re
from tika import parser
import warnings
from spacy.scorer import Scorer
from spacy.tokens import Doc
from spacy.training.example import Example

In [4]:
os.chdir('/content')

In [6]:
# !git clone https://github.com/Shavakchauhan/Resume_parser_using_deep_learning.git

Cloning into 'Resume_parser_using_deep_learning'...
remote: Enumerating objects: 324, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 324 (delta 7), reused 0 (delta 0), pack-reused 301
Receiving objects: 100% (324/324), 4.49 MiB | 17.87 MiB/s, done.
Resolving deltas: 100% (83/83), done.


In [6]:
spider_tagged_data = '/content/input'
list_tagged_files = os.listdir(spider_tagged_data)

In [7]:
# Code taken from https://github.com/explosion/spaCy/discussions/8456
def make_v3_dataset(data, db = []):
    nlp = spacy.blank('en')
    failed_record = []
    if not db:
        db = DocBin()
    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot['entities']:
            span = doc.char_span(start, end, label = label, alignment_mode = 'contract')
            if span is None:
                print(f'empty entity, {text}, {annot["entities"]}') #I expect this to never happen
            else:
                ents.append(span)
        try:
            doc.ents = ents
        except:
            failed_record.append((text, annot))
        db.add(doc)
    return db, failed_record

In [9]:
# Converting json annotated files to spacy format
for i in tqdm(range(len(list_tagged_files))):
  file_path = os.path.join(spider_tagged_data,list_tagged_files[i])
  # print(file_path)
  with open(file_path,'r') as f:
    data = json.load(f)
  saving_path = os.path.join('/content/output',list_tagged_files[i][:-5]+'.spacy')
  a,b = make_v3_dataset(data['annotations'])
  a.to_disk(saving_path)

100%|██████████| 1/1 [00:00<00:00,  6.56it/s]


In [10]:
!python -m spacy init fill-config /content/base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [11]:
!python -m spacy train -g 0 config.cfg  --output generated_model/ --paths.train /content/output --paths.dev cross_val/

✔ Created output directory: generated_model
ℹ Saving to output directory: generated_model
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-10-21 09:50:54,868] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-10-21 09:50:54,878] [INFO] Pipeline: ['transformer', 'ner']
INFO:spacy:Pipeline: ['transformer', 'ner']
[2022-10-21 09:50:54,882] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-10-21 09:50:54,883] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertF

In [12]:
nlp = spacy.load('generated_model/model-best')

In [28]:
main_str=['''bachelors of engineering - be percentage: - 60%  pillai hoc college of enginnering and technology from: - 2016-2020016  hsc - (12th) percentage: - 60%  smt sushiladevi deshmukh vidyalaya & jr college from: - 2014-2016  ssc- (10th) percentage: -79.8 %  smt sushiladevi deshmukh vidyalaya  79.80 ''']

In [29]:
for doc in nlp.pipe(main_str, disable=["tagger", "parser"]):
  for ent in doc.ents:
    print((ent.text,ent.label_))

('bachelors of engineering', 'DEGREE')
('pillai hoc college of enginnering and technology', 'SCHOOL')
('2016', 'YEAR')
('12th', 'DEGREE')
('sushiladevi deshmukh vidyalaya & jr college', 'SCHOOL')
('2014-2016', 'YEAR')
('10th', 'DEGREE')
